In [ ]:
from pathlib import Path

from pysr import PySRRegressor

# pysr before others, since it has to be before torch?
from jaxtyping import Float
import numpy as np
from zanj import ZANJ

from maze_dataset import MazeDatasetConfig
from maze_dataset.generation import LatticeMazeGenerators
from maze_dataset.benchmark.config_sweep import (
    SweepResult,
    full_percolation_analysis,
    dataset_success_fraction,
    plot_grouped,
)

In [ ]:
# magic auto-reload
%load_ext autoreload
%autoreload

In [ ]:
# Run the analysis
results: SweepResult = SweepResult.analyze(
    configs=[
        MazeDatasetConfig(
            name=f"g{grid_n}-perc",
            grid_n=grid_n,
            n_mazes=32,
            maze_ctor=LatticeMazeGenerators.gen_percolation,
            maze_ctor_kwargs=dict(),
            endpoint_kwargs=dict(
                deadend_start=False,
                deadend_end=False,
                endpoints_not_equal=False,
                except_on_no_valid_endpoint=False,
            ),
        )
        for grid_n in [2, 4, 6]
    ],
    param_values=np.linspace(0.0, 1.0, 16).tolist(),
    param_key="maze_ctor_kwargs.p",
    analyze_func=dataset_success_fraction,
    parallel=False,
)

# Plot results
results.plot(save_path=None, cfg_keys=["n_mazes", "endpoint_kwargs"])

In [ ]:
path = Path("../tests/_temp/dataset_frac_sweep/results_small.zanj")
results.save(path)
ZANJ().read(path).plot(cfg_keys=["n_mazes", "endpoint_kwargs"])

In [ ]:
results_full: SweepResult = full_percolation_analysis(
    n_mazes=16,
    p_val_count=16,
    grid_sizes=[2, 4, 6],
    parallel=False,
)

In [ ]:
plot_grouped(results_full)

In [ ]:
def extract_training_data(
    sweep_result: SweepResult,
) -> tuple[Float[np.ndarray, "num_rows 5"], Float[np.ndarray, " num_rows"]]:
    """Extract data (X, y) from a SweepResult.

    # Parameters:
     - `sweep_result : SweepResult`
        The sweep result holding configs and success arrays.

    # Returns:
     - `X : Float[np.ndarray, "num_rows 5"]`
        Stacked [p, grid_n, deadends, endpoints_not_equal, generator_func] for each config & param-value
     - `y : Float[np.ndarray, "num_rows"]`
        The corresponding success rate
    """
    X_list: list[list[float]] = []
    y_list: list[float] = []
    for cfg in sweep_result.configs:
        # success_arr is an array of success rates for param_values
        success_arr = sweep_result.result_values[cfg.to_fname()]
        for i, p in enumerate(sweep_result.param_values):
            # Temporarily override p in the config's array representation:
            arr = cfg._to_ps_array().copy()
            arr[0] = p  # index 0 is 'p'
            X_list.append(arr)
            y_list.append(success_arr[i])

    return np.array(X_list, dtype=np.float64), np.array(y_list, dtype=np.float64)

In [ ]:

# RESULTS_LOADED: SweepResult = SweepResult.read("../docs/benchmarks/percolation_fractions/medium/result-n128-c42-p50.zanj")
RESULTS_LOADED: SweepResult = SweepResult.read("../docs/benchmarks/percolation_fractions/small/result-n64-c30-p25.zanj")
# RESULTS_LOADED: SweepResult = SweepResult.read("../docs/benchmarks/percolation_fractions/test/result-n16-c12-p16.zanj")

# Convert to arrays
X, y = extract_training_data(RESULTS_LOADED)

# Fit the PySR model
model = PySRRegressor(
    niterations=100,
    # Adjust operators as needed
    unary_operators=[
		"exp",
		"log",
        "square(x) = x^2",
        "cube(x) = x^3",
        "sigmoid(x) = 1/(1 + exp(-x))",
    ],
    binary_operators=["+", "-", "*", "/"],
    # If you want advanced features, specify them here:
    # e.g. select_k_features=4, population_size=1000, etc.
    progress=True,
    model_selection="best",
)
model.fit(X, y)



In [ ]:


# Create a callable that predicts from MazeDatasetConfig
predict_fn = model.get_best()["lambda_format"]
print(f"Best PySR Equation: {model.get_best()['equation'] = }")
print(f"{predict_fn =}")

def predict_config(cfg: MazeDatasetConfig) -> float:
    arr = cfg._to_ps_array()
    result = predict_fn(arr)[0]
    return float(result)  # pass the array as separate args

# Example new config
test_config = MazeDatasetConfig(
    name="some_test",
    grid_n=16,
    n_mazes=100,
    maze_ctor=LatticeMazeGenerators.gen_percolation,
    maze_ctor_kwargs={"p": 0.3},
    endpoint_kwargs={
        "deadend_start": True,
        "deadend_end": True,
        "endpoints_not_equal": True,
        "except_on_no_valid_endpoint": False,
    },
)
predicted_rate = predict_config(test_config)
print("Best PySR Equation:", model.get_best()["equation"])
print("Predicted success rate for test_config:", predicted_rate)


plot_grouped(RESULTS_LOADED, predict_fn=predict_config)